In [53]:
import requests

BASE_URL = "http://localhost:8000" 

def add_user(firebase_uid: str, name: str):
    user_model = {"firebase_uid": firebase_uid, "name": name}
    response = requests.post(f"{BASE_URL}/users/", json=user_model)
    return response.status_code, response.json()

def get_users():
    response = requests.get(f"{BASE_URL}/users/")
    return response.status_code, response.json()

def get_user(id: str):
    response = requests.get(f"{BASE_URL}/users/{id}")
    return response.status_code, response.json()

def update_user(id: str, name: str):
    response = requests.put(f"{BASE_URL}/users/{id}", json={"name": name})
    return response.status_code, response.json()

def delete_user(id: str):
    response = requests.delete(f"{BASE_URL}/users/{id}")
    return response.status_code

def get_games():
    response = requests.get(f"{BASE_URL}/games/")
    return response.status_code, response.json()

def create_game(name: str, game_type: str):
    response = requests.post(f"{BASE_URL}/games/", json={"name": name, "type": game_type})
    return response.status_code, response.json()

def add_user_to_game(game_id: str, user_id: str):
    response = requests.patch(f"{BASE_URL}/games/{game_id}/add_user/{user_id}")
    return response.status_code, response.json()

def remove_user_from_game(game_id: str, user_id: str):
    response = requests.patch(f"{BASE_URL}/games/{game_id}/remove_user/{user_id}")
    return response.status_code, response.json()

def start_game(game_id: str):
    response = requests.patch(f"{BASE_URL}/games/{game_id}/start")
    return response.status_code





In [8]:
print(add_user("12345test","ethantest"))

(201, {'id': '682e907f9ead523f1a672ee0', 'firebase_uid': '12345test', 'name': 'ethantest', 'games': 0, 'wins': 0, 'username_set': True})


In [33]:
print(get_users())

(200, {'users': [{'id': '681fdb709d14dcc91d530575', 'firebase_uid': 'dJ84cRICX2MtkzOi9ceaK333QPQ2', 'name': 'lordboba', 'games': 0, 'wins': 0, 'username_set': True}, {'id': '6826db221601e717a6843e86', 'firebase_uid': 'abc123firebase', 'name': 'cool_user', 'games': 0, 'wins': 0, 'username_set': True}, {'id': '6826dce6e8bce8f5a061dfdc', 'firebase_uid': 'test-uid-123', 'name': None, 'games': 0, 'wins': 0, 'username_set': False}, {'id': '6826dd2b1601e717a6843e87', 'firebase_uid': 'hkrwtgoIy7bJRttAASiFyLCtkUs1', 'name': 'ethan', 'games': 0, 'wins': 0, 'username_set': True}, {'id': '68277f750a9dcc37ba836316', 'firebase_uid': 'abc456firebase', 'name': 'cool_user2', 'games': 0, 'wins': 0, 'username_set': True}]})


In [16]:
print(get_user("682e907f9ead523f1a672ee0"))

(200, {'id': '682e907f9ead523f1a672ee0', 'firebase_uid': '12345test', 'name': 'ethantest', 'games': 0, 'wins': 0, 'username_set': True})


In [18]:
print(update_user("682e907f9ead523f1a672ee0","ethantest2"))

(200, {'id': '682e907f9ead523f1a672ee0', 'firebase_uid': '12345test', 'name': 'ethantest2', 'games': 0, 'wins': 0, 'username_set': True})


In [22]:
print(delete_user("682e907f9ead523f1a672ee0"))

(404, {'detail': 'User 682e907f9ead523f1a672ee0 not found'})


In [55]:
print(get_games())

(200, {'games': []})


In [31]:
print(create_game("ethantestgame", "simple"))

(200, {'id': '682e9cda9ead523f1a672ee1', 'name': 'ethantestgame', 'type': 'simple', 'players': []})


In [49]:
print(add_user_to_game("682e9cda9ead523f1a672ee1","6826db221601e717a6843e86"))

(200, {'id': '682e9cda9ead523f1a672ee1', 'name': 'ethantestgame', 'type': 'simple', 'players': ['681fdb709d14dcc91d530575', '6826db221601e717a6843e86']})


In [46]:
print(remove_user_from_game("682e9cda9ead523f1a672ee1","681fdb709d14dcc91d530575"))

(200, {'id': '682e9cda9ead523f1a672ee1', 'name': 'ethantestgame', 'type': 'simple', 'players': []})


In [54]:
print(start_game("682e9cda9ead523f1a672ee1"))

404
